In [311]:
import pandas as pd
df = pd.read_json('debunk.json', encoding='utf-8')

In [312]:
df.head()

,Claim,Country,Date,Debunk_Date,Explaination,Factcheck_Org,Label,Link,Source,Source_Lang
0,\n Chinese state media claims Ghana’s coronavi...,Ghana,2020-04-13,2020/04/13,Official Ghana government data contradicts the...,GhanaFact,FALSE,https://www.poynter.org/?ifcn_misinformation=c...,https://ghanafact.com/false-chinese-state-medi...,en
1,\n Video: Migrant labourers in Kerala gambling...,India,2020-04-11,2020/04/11,An old and unrelated video is used to make the...,FactCrescendo,FALSE,https://www.poynter.org/?ifcn_misinformation=v...,https://www.malayalam.factcrescendo.com/old-an...,ml
2,\n Coronavirus can be avoided by eating nigell...,India,2020-04-11,2020/04/11,Hydroxychloroquine is not present in the subst...,Newschecker,FALSE,https://www.poynter.org/?ifcn_misinformation=c...,https://www.newschecker.in/article/news-detail...,hi
3,\n ICMR issued a report claiming the number of...,India,2020-04-11,2020/01/01,ICMR did not publish any such report about the...,Vishvas News,FALSE,https://www.poynter.org/?ifcn_misinformation=i...,https://www.facebook.com/groups/dsdrss/permali...,en
4,\n Sanitize masks by microwaving them for 2-3 ...,Colombia,2020-04-11,2020/04/15,Fabric masks should not be sanitized by microw...,Vishvas News,FALSE,https://www.poynter.org/?ifcn_misinformation=s...,https://www.vishvasnews.com/english/health/fac...,hi


In [313]:
df_new = df[df.Source_Lang.eq('en')].copy()
df_new['Claim'] = df_new['Claim'].str.replace(r'[-:,.;:%\s+\d+]',' ')
df_new.reindex()
df_new.head()

,Claim,Country,Date,Debunk_Date,Explaination,Factcheck_Org,Label,Link,Source,Source_Lang
0,Chinese state media claims Ghana’s coronavir...,Ghana,2020-04-13,2020/04/13,Official Ghana government data contradicts the...,GhanaFact,FALSE,https://www.poynter.org/?ifcn_misinformation=c...,https://ghanafact.com/false-chinese-state-medi...,en
3,ICMR issued a report claiming the number of ...,India,2020-04-11,2020/01/01,ICMR did not publish any such report about the...,Vishvas News,FALSE,https://www.poynter.org/?ifcn_misinformation=i...,https://www.facebook.com/groups/dsdrss/permali...,en
5,Hindu man in Rajasthan killed by Muslims ami...,India,2020-04-11,2020/04/11,The video is originally from Pakistan where a ...,FactCrescendo,FALSE,https://www.poynter.org/?ifcn_misinformation=h...,https://www.factcrescendo.com/fact-check-hindu...,en
10,A WhatsApp forward along with a video is d...,India,2020-04-11,2020/04/11,The video being circulated in the name of Tabl...,Newschecker,FALSE,https://www.poynter.org/?ifcn_misinformation=a...,https://www.newschecker.in/article/news-detail...,en
14,Industrialist Ratan Tata’s view on economic ...,India,2020-04-11,2020/04/11,The quote on economical downfall due to COVID-...,FactCrescendo,FALSE,https://www.poynter.org/?ifcn_misinformation=i...,https://english.factcrescendo.com/2020/04/11/f...,en


In [340]:
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
import string
import gensim
from nltk.tokenize import sent_tokenize

porter = PorterStemmer()
lan_porter = LancasterStemmer()
stop_words = set(stopwords.words('english'))
clean_content=[]
i = 0
for content in df_new['Claim']:
    #turn string into tokens and remove \n and \r tags
    tokenizer = nltk.RegexpTokenizer('\s+', gaps=True)
    word_tokens = tokenizer.tokenize(content)
    words = [porter.stem(w.lower()) for w in word_tokens if w not in stop_words and w.isalpha()]
    #lowercase, remove stop words and punctuation/numbers
    clean_content.append( [w for w in words if len(w) > 3] ) 
clean_content

[['chines',
  'state',
  'media',
  'claim',
  'coronaviru',
  'case',
  'surpass',
  'death'],
 ['icmr',
  'issu',
  'report',
  'claim',
  'number',
  'peopl',
  'infect',
  'thte',
  'coronaviru',
  'increas',
  'lakh',
  'india',
  'announc',
  'lockdown'],
 ['hindu', 'rajasthan', 'kill', 'muslim', 'amidst', 'lockdown'],
 ['whatsapp',
  'forward',
  'along',
  'video',
  'round',
  'claim',
  'person',
  'belong',
  'tablighi',
  'jamaat',
  'kept',
  'isol',
  'ward',
  'start',
  'wander',
  'nake',
  'viral',
  'forward',
  'also',
  'claim',
  'person',
  'wander',
  'nake',
  'vulgar',
  'intent'],
 ['industrialist', 'ratan', 'view', 'econom', 'downfal', 'covid'],
 ['mustard', 'kill', 'coronaviru'],
 ['photograph',
  'rail',
  'track',
  'colombo',
  'fill',
  'wild',
  'flower',
  'current',
  'curfew',
  'period',
  'lanka'],
 ['vandana',
  'tiwari',
  'part',
  'team',
  'investig',
  'identifi',
  'attende',
  'tablighi',
  'jamaat',
  'bareilli',
  'attack',
  'succumb',


In [341]:
from gensim.utils import simple_preprocess

#alternative way
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def strip_newline(series):
    return [review.replace('\n','') for review in series]

df['Claim'] = strip_newline(df['Claim'])
words = list(sent_to_words(df['Claim']))
words = remove_stopwords(words)
bigram = bigrams(words)
bigram = [bigram[review] for review in words]
words = sent_to_words(m)

In [342]:
import gensim
import gensim.corpora as corpora
dictionary = gensim.corpora.Dictionary(clean_content)
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [343]:
bow_corpus = [dictionary.doc2bow(doc) for doc in clean_content]

In [344]:
for i in range(len(bow_corpus[0])):
    print("Word {} (\"{}\") appears {} time.".format(bow_corpus[0][i][0], 
                                                     dictionary[bow_corpus[0][i][0]], 
                                                     bow_corpus[0][i][1]))

Word 0 ("case") appears 1 time.
Word 1 ("chines") appears 1 time.
Word 2 ("death") appears 1 time.
Word 3 ("media") appears 1 time.
Word 4 ("state") appears 1 time.


In [345]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 20, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [346]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.236*"viru" + 0.152*"minist" + 0.123*"prime" + 0.094*"order" + 0.080*"issu" + 0.072*"indian" + 0.032*"chines" + 0.025*"protect" + 0.023*"countri" + 0.021*"presid"


Topic: 1 
Words: 0.542*"infect" + 0.071*"philippin" + 0.068*"nation" + 0.036*"kill" + 0.029*"hospit" + 0.029*"china" + 0.025*"govern" + 0.025*"indian" + 0.023*"lanka" + 0.021*"health"


Topic: 2 
Words: 0.156*"spread" + 0.129*"video" + 0.080*"view" + 0.077*"caus" + 0.064*"facebook" + 0.064*"time" + 0.055*"quarantin" + 0.053*"alongsid" + 0.050*"thousand" + 0.047*"youtub"


Topic: 3 
Words: 0.258*"water" + 0.146*"drink" + 0.093*"gargl" + 0.093*"salt" + 0.087*"cure" + 0.086*"close" + 0.070*"prevent" + 0.056*"kill" + 0.027*"viru" + 0.018*"help"


Topic: 4 
Words: 0.154*"post" + 0.139*"facebook" + 0.134*"share" + 0.100*"time" + 0.078*"multipl" + 0.072*"twitter" + 0.062*"thousand" + 0.039*"alongsid" + 0.025*"hundr" + 0.022*"photo"


Topic: 5 
Words: 0.322*"india" + 0.147*"high" + 0.113*"stop" + 0.083*"offici" + 

In [349]:
m = lda_model.get_document_topics(bow_corpus[5])
m.sort(key=lambda x: x[1], reverse=True)
m[1]

(0, 0.025000019)

In [348]:
df_new[:6]

,Claim,Country,Date,Debunk_Date,Explaination,Factcheck_Org,Label,Link,Source,Source_Lang
0,Chinese state media claims Ghana’s coronavir...,Ghana,2020-04-13,2020/04/13,Official Ghana government data contradicts the...,GhanaFact,FALSE,https://www.poynter.org/?ifcn_misinformation=c...,https://ghanafact.com/false-chinese-state-medi...,en
3,ICMR issued a report claiming the number of ...,India,2020-04-11,2020/01/01,ICMR did not publish any such report about the...,Vishvas News,FALSE,https://www.poynter.org/?ifcn_misinformation=i...,https://www.facebook.com/groups/dsdrss/permali...,en
5,Hindu man in Rajasthan killed by Muslims ami...,India,2020-04-11,2020/04/11,The video is originally from Pakistan where a ...,FactCrescendo,FALSE,https://www.poynter.org/?ifcn_misinformation=h...,https://www.factcrescendo.com/fact-check-hindu...,en
10,A WhatsApp forward along with a video is d...,India,2020-04-11,2020/04/11,The video being circulated in the name of Tabl...,Newschecker,FALSE,https://www.poynter.org/?ifcn_misinformation=a...,https://www.newschecker.in/article/news-detail...,en
14,Industrialist Ratan Tata’s view on economic ...,India,2020-04-11,2020/04/11,The quote on economical downfall due to COVID-...,FactCrescendo,FALSE,https://www.poynter.org/?ifcn_misinformation=i...,https://english.factcrescendo.com/2020/04/11/f...,en
19,Mustard oil can kill coronavirus,India,2020-04-10,2020/04/10,"No, mustard oil cannot cure coronavirus; howev...",Vishvas News,FALSE,https://www.poynter.org/?ifcn_misinformation=m...,https://www.vishvasnews.com/english/health/fac...,en
